In [26]:
import numpy as np
import pandas as pd
import torch
import wandb
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report
from torch import optim, nn
import torchmetrics
from tqdm import tqdm
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger, WandbLogger, CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

wandb.login()

if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
    device = "cuda"
else:
    device = "cpu"
print(f"Using {device} device")


PyTorch version: 2.1.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 2080
Using cuda device


In [27]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'input_size': {
            'values': [221]
        },
        'hidden_size': {
            'values': [128, 256, 512, 1024]
        },
        'num_layers': {
            'min': 2,
            'max': 15
        },
        'dropout_prob': {
            'values': [0.2, 0.3, 0.4, 0.5]
        },
        'activation': {
            'values': ['ReLU']
        },
        'decrease_size': {
            'values': [False, True]
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
        'lr': {
            'min': 1e-5,
            'max': 1e-1
        },
        'weight_decay': {
            'values': [1e-5]
        },
        'max_epochs': {
            'values': [200]
        },
        'patience': {
            'values': [10, 20, 30, 40]
        },
        'merged': {
            'values': [True]
        }
    }
}

In [28]:
sweep_id = wandb.sweep(sweep_config, project='leaguify')

Create sweep with ID: tih0fcwn
Sweep URL: https://wandb.ai/moritz-palm/leaguify/sweeps/tih0fcwn


In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, output_size=1, activation=nn.ReLU(),
                 decrease_size=False):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential()
        self.linear_relu_stack.append(nn.Linear(input_size, hidden_size))
        for i in range(num_layers - 1):
            if decrease_size:
                next_hidden_size = int(self.hidden_size // 2)
            else:
                next_hidden_size = self.hidden_size
            self.linear_relu_stack.append(self.dropout)
            self.linear_relu_stack.append(nn.BatchNorm1d(self.hidden_size))
            self.linear_relu_stack.append(nn.Linear(self.hidden_size, next_hidden_size))
            self.linear_relu_stack.append(activation)
            self.hidden_size = next_hidden_size
        self.linear_relu_stack.append(nn.Linear(self.hidden_size, self.output_size))
        self.linear_relu_stack.append(nn.Sigmoid())

    def forward(self, x):
        """

        :param x:
        :return:
        """
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [30]:
class LNN(L.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, output_size=1, activation=nn.ReLU(),
                 decrease_size=False):
        super().__init__()
        self.model = NeuralNetwork(input_size, hidden_size, num_layers, dropout_prob, output_size, activation,
                                   decrease_size)
        self.criterion = nn.BCELoss()
        self.save_hyperparameters()
        self.accuracy = torchmetrics.classification.BinaryAccuracy()
        self.f1 = torchmetrics.classification.BinaryF1Score()
        self.confusion_matrix = torchmetrics.classification.BinaryConfusionMatrix()

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        y_hat = self.model(x).squeeze(-1)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', self.accuracy(y_hat, y), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        y_hat = self.model(x).squeeze(-1)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.accuracy(y_hat, y), prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        y_hat = self.model(x).squeeze(-1)
        print(f'y_shape: {y.shape}, y_hat_shape: {y_hat.shape}')
        loss = self.criterion(y_hat, y)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', self.accuracy(y_hat, y), prog_bar=True)
        self.log('test_f1', self.f1(y_hat, y), prog_bar=True)
        print(f'test_confusion_matrix {self.confusion_matrix(y_hat, y)}')
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [31]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, )
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64)
        self.transform = transform
        self.target_transform = target_transform
        print(f'labels: {self.labels}')
        self.print_statistics()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx, 1:]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

    def print_statistics(self):
        print(f'Number of samples: {len(self.data)}')
        print(f'Number of features: {len(self.data[0])}')
        print(f'Number of labels: {len(self.labels)}')
        print(f'Number of classes: {len(np.unique(self.labels.cpu().numpy()))}')
        print(f'Number of samples per class: {np.bincount(self.labels.cpu().numpy())}')


In [32]:
def train(config=None):
    """

    :param config:
    :return:
    """
    with wandb.init(config=config):
        data_dir = '../../data/static_11_12_23/processed'
        config = wandb.config
        model = LNN(config.input_size, config.hidden_size, config.num_layers, config.dropout_prob)

        wandb_logger = WandbLogger()
        wandb_logger.watch(model)
        # training_data = wandb.Artifact('training_data', type='dataset')
        # training_data.add_dir(data_dir)
        # wandb_logger.experiment.log_artifact(training_data)
        if config.merged:
            train_loader = DataLoader(StaticDataset(data_dir + '/train_static_merged.npy'),
                                      batch_size=config.batch_size,
                                      shuffle=True)
            val_loader = DataLoader(StaticDataset(data_dir + '/val_static_merged.npy'), batch_size=config.batch_size,
                                    shuffle=True)
            test_loader = DataLoader(StaticDataset(data_dir + '/test_static_merged.npy'), batch_size=config.batch_size,
                                     shuffle=True)
        else:
            train_loader = DataLoader(StaticDataset(data_dir + '/train_static.npy'), batch_size=config.batch_size,
                                      shuffle=True)
            val_loader = DataLoader(StaticDataset(data_dir + '/val_static.npy'), batch_size=config.batch_size,
                                    shuffle=True)
            test_loader = DataLoader(StaticDataset(data_dir + '/test_static.npy'), batch_size=config.batch_size,
                                     shuffle=True)
        trainer = L.Trainer(max_epochs=config.max_epochs, accelerator=device, devices=1,
                            logger=wandb_logger,
                            callbacks=[
                                EarlyStopping(monitor='val_acc', patience=config.patience, verbose=True, mode='max'),
                                ModelCheckpoint(monitor='val_acc', dirpath='models', filename='model', save_top_k=2,
                                                mode='max',
                                                every_n_epochs=1)
                            ])
        trainer.fit(model, train_loader, val_loader)
        trainer.test(model, test_loader)


In [33]:
wandb.agent(sweep_id, function=train, count=2)

wandb: Agent Starting Run: nw7ocoom with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 64
wandb: 	decrease_size: False
wandb: 	dropout_prob: 0.2
wandb: 	hidden_size: 256
wandb: 	input_size: 221
wandb: 	lr: 0.05052929108760294
wandb: 	max_epochs: 200
wandb: 	merged: True
wandb: 	num_layers: 14
wandb: 	patience: 10
wandb: 	weight_decay: 1e-05


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


labels: tensor([1, 1, 0,  ..., 0, 1, 0])
Number of samples: 31243
Number of features: 222
Number of labels: 31243
Number of classes: 2
Number of samples per class: [15084 16159]
labels: tensor([0, 1, 1,  ..., 1, 1, 0])
Number of samples: 3472
Number of features: 222
Number of labels: 3472
Number of classes: 2
Number of samples per class: [1676 1796]
labels: tensor([0, 0, 0,  ..., 1, 1, 0])
Number of samples: 3858
Number of features: 222
Number of labels: 3858
Number of classes: 2
Number of samples per class: [1863 1995]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                  | Params
-----------------------------------------------------------
0 | model            | NeuralNetwork         | 919 K 
1 | criterion        | BCELoss               | 0     
2 | accuracy         | BinaryAccuracy        | 0     
3 | f1               | BinaryF1Score         | 0     
4 | confusion_matrix | BinaryConfusionMatrix | 0     
-----------------------------------------------------------
919 K     Trainable params
0         Non-trainable params
919 K     Total params
3.676     Total estimated model params size (MB)
wandb: WARNING Config item 'input_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_layers' was locked by 'sweep' (ignored

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_acc did not improve in the last 10 records. Best score: 0.700. Signaling Trainer to stop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'input_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_layers' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dropout_prob' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'activation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'decrease_size' was locked by 'sweep' (ignored update).


Testing: |          | 0/? [00:00<?, ?it/s]

y_shape: torch.Size([256]), y_hat_shape: torch.Size([256])
test_confusion_matrix tensor([[83, 42],
        [36, 95]], device='cuda:0')
y_shape: torch.Size([256]), y_hat_shape: torch.Size([256])
test_confusion_matrix tensor([[79, 39],
        [40, 98]], device='cuda:0')
y_shape: torch.Size([256]), y_hat_shape: torch.Size([256])
test_confusion_matrix tensor([[83, 48],
        [40, 85]], device='cuda:0')
y_shape: torch.Size([256]), y_hat_shape: torch.Size([256])
test_confusion_matrix tensor([[ 87,  36],
        [ 31, 102]], device='cuda:0')
y_shape: torch.Size([256]), y_hat_shape: torch.Size([256])
test_confusion_matrix tensor([[83, 41],
        [33, 99]], device='cuda:0')
y_shape: torch.Size([256]), y_hat_shape: torch.Size([256])
test_confusion_matrix tensor([[84, 48],
        [25, 99]], device='cuda:0')
y_shape: torch.Size([256]), y_hat_shape: torch.Size([256])
test_confusion_matrix tensor([[86, 36],
        [40, 94]], device='cuda:0')
y_shape: torch.Size([256]), y_hat_shape: torch.Size

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█
test_acc,▁
test_f1,▁
test_loss,▁
train_acc,▁▇▄▅▃▃▅▄▄▃▅▁▂▇▅▅▅▅▅▂▅▅▅▃█▇▅▇▂█▅
train_loss,▇▂▅▃▅▅▄▃▄▅▃█▄▃▄▄▅▃▄▆▂▄▃▅▁▂▃▁▇▁▄
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
val_acc,▁▅█▂▄▂▅▅▆▆▃▁▆
val_loss,█▇█▄▄█▇▅█▄▆▁▄
epoch,13
test_acc,0.70425


In [34]:
import os

print(os.getcwd())

C:\Users\morit\Documents\Informatik\Projekte\LeagueOfLegendsWinPrediction\Training\static
